## Use pandas to process data using several basic operations

In [91]:
import numpy as np
import pandas as pd

### Update column names and Obtain the set of unique values of a particular column

In [130]:
# read input file
input_file = 'data/pandas_data_example.csv'
orders = pd.read_csv(input_file, encoding='utf-8-sig')

# update column name
newColumns = {'Departure': 'Departure_Time', 'Arrival': 'Arrival_Time'}
orders.rename(columns=newColumns, inplace=True)
print '\nColumns names: ', orders.columns

# print the set of unique values of a particular column
originSet = orders['Origin'].unique()
destinationSet = orders['Destination'].unique()
print '\nOrigin Set:', originSet
print '\nDestination Set:', destinationSet
print '\nExample rows of original data:\n', orders.iloc[0:4]

 
Columns names:  Index([u'Origin', u'Destination', u'Quantity', u'Departure_Time',
       u'Arrival_Time'],
      dtype='object')

Origin Set: [u'AA' u'BB' u'CC' u'DD']

Destination Set: [u'ABC' u'DEF' u'GHI' u'JKL' u'MNO' u'PQR' u'VWX' u'STU']

Example rows of original data:
  Origin Destination  Quantity Departure_Time   Arrival_Time
0     AA         ABC        24    1/3/17 4:00   1/5/17 23:24
1     AA         ABC        24    1/3/17 4:00   1/5/17 23:24
2     AA         ABC        24    1/3/17 4:00   1/5/17 23:24
3     AA         ABC        12    1/3/17 4:00  1/19/17 15:42


### Iterate through the DataFrame

In [135]:
for index, row in orders.iterrows():
    if row['Destination'] == 'GHI':
        print index, row['Origin'], row['Destination'], row['Quantity']

384 AA GHI 272
385 BB GHI 198
386 BB GHI 168
387 BB GHI 60
388 BB GHI 24
389 CC GHI 66
390 CC GHI 6
391 CC GHI 54
392 CC GHI 120
393 CC GHI 187
394 CC GHI 114
395 CC GHI 60
396 CC GHI 90
397 CC GHI 402
398 CC GHI 58
399 CC GHI 18


### Obtain the origins with the largest quantities

In [128]:
ordersByOrigin = orders.groupby(by=['Origin'])['Quantity'].sum().reset_index()
ordersByOrigin.sort_values(by=['Quantity'], ascending=False, inplace=True)
origins = ordersByOrigin.head(3)['Origin']
print origins.values

['CCCC' 'DDDD' 'BBBB']


### Apply function to a particular column

In [93]:
## apply function to a particular column
newOriginSet = ['AAAA', 'BBBB', 'CCCC', 'DDDD']
originMap = {x : y for (x, y) in zip(originSet, newOriginSet)} # dict(zip(originSet, newOriginSet))
orders['Origin'] = orders['Origin'].apply(lambda x: originMap[x] if x in originMap else x)
print '\nData after updating column names:\n', orders.iloc[0:4]


Data after updating column names:
  Origin Destination  Quantity Departure_Time   Arrival_Time
0   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24
1   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24
2   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24
3   AAAA         ABC        12    1/3/17 4:00  1/19/17 15:42


### Filter data under some criteria

In [94]:
## filter data
destionationSet = ['ABC', 'DEF', 'GHI', 'JKL', 'MNO', 'PQR', 'STU']
orders = orders.loc[orders['Destination'].isin(destinationSet)]
orders = orders.loc[orders['Quantity']>=5]

print '\nData after filtering:\n', orders.iloc[0:4]


Data after filtering:
  Origin Destination  Quantity Departure_Time   Arrival_Time
0   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24
1   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24
2   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24
3   AAAA         ABC        12    1/3/17 4:00  1/19/17 15:42


### Operate on Datatimelike columns

In [95]:
## operate on Datatimelike columns
departureTime = pd.to_datetime(orders['Departure_Time'], errors='coerce')
arrivalTime = pd.to_datetime(orders['Arrival_Time'], errors='coerce')
travelTime = (arrivalTime - departureTime).astype('timedelta64[s]')/(3600.0*24.0)
print '\nDeparture time series:\n', departureTime[0:4]


Departure time series:
0   2017-01-03 04:00:00
1   2017-01-03 04:00:00
2   2017-01-03 04:00:00
3   2017-01-03 04:00:00
Name: Departure_Time, dtype: datetime64[ns]


### Add columns to the DataFrame

In [96]:
## add columns to the DataFrame
orders.loc[:, 'Travel_Time'] = travelTime
orders.loc[:, 'Departure_Date'] = departureTime.dt.date # just keep the date part
orders.loc[:, 'Departure_Week'] = departureTime.dt.week # the week of year, index starting from 1
orders.loc[:, 'Departure_DayOfWeek'] = departureTime.dt.dayofweek # the day of week, e.g., 0-Mon, 4-Thu
orders.loc[:, 'Arrival_Date'] = arrivalTime.dt.date
orders.loc[:, 'Arrival_Week'] = arrivalTime.dt.week
orders.loc[:, 'Arrival_DayOfWeek'] = arrivalTime.dt.dayofweek
dayOfWeek = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
orders['Departure_DayOfWeek'] = orders['Departure_DayOfWeek'].apply(lambda x : dayOfWeek[x])
orders['Arrival_DayOfWeek'] = orders['Arrival_DayOfWeek'].apply(lambda x : dayOfWeek[x])

print '\nData after adding columns:\n', orders.iloc[0:4]


Data after adding columns:
  Origin Destination  Quantity Departure_Time   Arrival_Time  Travel_Time  \
0   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24     2.808333   
1   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24     2.808333   
2   AAAA         ABC        24    1/3/17 4:00   1/5/17 23:24     2.808333   
3   AAAA         ABC        12    1/3/17 4:00  1/19/17 15:42    16.487500   

  Departure_Date  Departure_Week Departure_DayOfWeek Arrival_Date  \
0     2017-01-03               1                 Tue   2017-01-05   
1     2017-01-03               1                 Tue   2017-01-05   
2     2017-01-03               1                 Tue   2017-01-05   
3     2017-01-03               1                 Tue   2017-01-19   

   Arrival_Week Arrival_DayOfWeek  
0             1               Thu  
1             1               Thu  
2             1               Thu  
3             3               Thu  


### Sort w.r.t. columns, (1) default order; (2) customized order

In [103]:
## sort w.r.t. columns in default order: numeric/time/alphabetical
orders.sort_values(by=['Origin', 'Destination', 'Departure_Time'], inplace=True)
print '\nData after sorting:\n', orders[['Origin','Destination','Quantity','Departure_Time','Arrival_Time']]

## sort w.r.t. columns in customized order:
dayOfWeek = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
sortedOrders = orders.groupby(by=['Origin','Destination','Departure_DayOfWeek'])['Quantity'].sum().reset_index()
# order by Departure_DayOfWeek
sortedOrders['Departure_DayOfWeek'] = sortedOrders['Departure_DayOfWeek'].astype('category')
sortedOrders['Departure_DayOfWeek'].cat.set_categories(dayOfWeek, inplace=True)
sortedOrders.sort_values(by=['Origin','Destination','Departure_DayOfWeek'], ascending=[True, True, True], inplace=True)
print '\nData after sorting:\n', sortedOrders


Data after sorting:
    Origin Destination  Quantity Departure_Time   Arrival_Time
9     AAAA         ABC        54   1/10/17 4:00  1/12/17 20:09
10    AAAA         ABC        12   1/10/17 4:00  1/26/17 20:10
11    AAAA         ABC        18   1/10/17 4:00  1/12/17 20:09
12    AAAA         ABC        50   1/16/17 4:00  1/26/17 20:10
13    AAAA         ABC        84   1/17/17 4:00  1/19/17 15:42
14    AAAA         ABC        24   1/17/17 4:00  1/19/17 15:42
15    AAAA         ABC        24   1/17/17 4:00  1/26/17 20:10
16    AAAA         ABC        24   1/17/17 4:00  1/19/17 15:42
17    AAAA         ABC        32   1/18/17 4:00  1/26/17 20:10
18    AAAA         ABC       168   1/23/17 4:00  1/26/17 20:10
19    AAAA         ABC         6   1/24/17 4:00  1/26/17 20:10
20    AAAA         ABC       188   1/24/17 4:00  1/26/17 20:10
21    AAAA         ABC       647   1/24/17 4:00  1/26/17 20:10
22    AAAA         ABC         6   1/25/17 3:10    2/4/17 7:57
23    AAAA         ABC        16  

### Groupby functions: (1) single groupby; (2) multiple groupby

In [98]:
# use of single groupby function
ordersByOriDesPair = orders.groupby(by=['Origin', 'Destination'])['Quantity'].sum().reset_index()
print '\nData after group by single function:\n', ordersByOriDesPair

# use of multiple groupby functions
groupByFuns = {'Quantity': ['sum'], 'Travel_Time': ['mean', 'std']}
ordersByMultiGroupby = orders.groupby(by=['Origin', 'Destination']).agg(groupByFuns).reset_index()
ordersByMultiGroupby.fillna(0.0, inplace=True)

# # drop the outermost level from the hierarchical column index
# ordersByMultiGroupby.columns = ordersByMultiGroupby.columns.droplevel(0)

# form multi-level column labels
newLabels = ['_'.join(col) for col in ordersByMultiGroupby.columns.ravel()]
ordersByMultiGroupby.columns = newLabels
print '\nData after group by multiple functions:\n', ordersByMultiGroupby


Data after group by single function:
   Origin Destination  Quantity
0    AAAA         ABC      3895
1    AAAA         DEF        98
2    AAAA         GHI       272
3    AAAA         MNO       105
4    AAAA         PQR       221
5    AAAA         STU       616
6    AAAA         VWX      2782
7    BBBB         ABC      8084
8    BBBB         DEF       136
9    BBBB         GHI       450
10   BBBB         JKL      1138
11   BBBB         MNO       230
12   BBBB         PQR       716
13   BBBB         STU      1068
14   BBBB         VWX      3015
15   CCCC         ABC     18908
16   CCCC         DEF        15
17   CCCC         GHI      1175
18   CCCC         JKL      1944
19   CCCC         MNO      1820
20   CCCC         PQR      6201
21   CCCC         STU       976
22   CCCC         VWX     10196
23   DDDD         ABC      8160
24   DDDD         DEF       482
25   DDDD         JKL      3565
26   DDDD         MNO       132
27   DDDD         PQR       840
28   DDDD         STU      2339
29

### Unstack function

In [122]:
## unstack function
dayOfWeek = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
ordersByOrigin = orders.groupby(by=['Origin', 'Departure_Week', 'Departure_DayOfWeek'])['Quantity'].sum().reset_index()

# order by Departure_DayOfWeek first
ordersByOrigin['Departure_DayOfWeek'] = ordersByOrigin['Departure_DayOfWeek'].astype('category')
ordersByOrigin['Departure_DayOfWeek'].cat.set_categories(dayOfWeek, inplace=True)

# unstack by Departure_DayOfWeek
unstackByDayofweek = ordersByOrigin.set_index(['Origin', 'Departure_Week', 'Departure_DayOfWeek'])
unstackByDayofweek = unstackByDayofweek.unstack('Departure_DayOfWeek')
unstackByDayofweek.fillna(0.0, inplace=True)
print '\nData unstacked by DayOfWeek:\n', unstackByDayofweek

# unstack by Departure_Week
unstackByWeek = ordersByOrigin.set_index(['Origin', 'Departure_DayOfWeek', 'Departure_Week'])
unstackByWeek = unstackByWeek.unstack('Departure_Week')
unstackByWeek.fillna(0.0, inplace=True)
print '\nData unstacked by Week:\n', unstackByWeek


Data unstacked by DayOfWeek:
                      Quantity                              
Departure_DayOfWeek        Mon     Tue    Wed     Thu    Fri
Origin Departure_Week                                       
AAAA   1                   0.0   240.0  144.0   120.0    0.0
       2                 462.0    84.0    0.0    50.0    0.0
       3                 794.0   156.0   32.0    37.0    0.0
       4                 814.0   849.0   46.0    72.0    0.0
       5                 406.0     0.0    0.0   115.0    0.0
       6                 514.0   777.0  272.0   117.0    0.0
       7                 432.0   229.0  253.0    40.0    0.0
       8                 474.0   204.0  204.0    37.0   15.0
BBBB   1                   0.0   482.0  288.0   374.0    0.0
       2                 814.0   348.0   84.0   118.0    0.0
       3                1134.0   393.0  172.0   135.0   60.0
       4                1247.0   869.0  232.0   146.0   18.0
       5                1079.0     0.0   74.0   107.0  